In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# COVID-19 in Limeira-SP, Brazil - Basic Analisys

**Last update: 04/10/2021 12:32 PM**

## Introduction

<img src="https://upload.wikimedia.org/wikipedia/commons/4/46/SaoPaulo_Municip_Limeira.svg" alt="Map of Limeira" width="400"/>

The city of [Limeira](https://pt.wikipedia.org/wiki/Limeira) is located at Sao Paulo state, in Brazil. The city has an area of 518.0 km² and has slightly more than 300,000 habitants (acordingly to the last census, in 2017).

The first cases of coronavirus was detected in the city by the end of March, 2020. As the pandemic goes, the city hall built the URC (Coronavirus Reference Unit, a specialized hospital for treat COVID patients).

Using the dataset of public COVID pandemic data, the goal of this basic analisys is to make some visualizations that could help the citizens to better understand the evolution coronavirus at the city.

**NOTICE: This notebook will be updated as new data are being reported.**

In [ ]:
data = pd.read_csv('/kaggle/input/covid19-in-limeiraspbrazil/covid-limeira-daily.csv')
data['Date'] = pd.to_datetime(data['Date'])

The table bellow shows the numbers reported for last week.

In [ ]:
data.tail(10)

## Daily Growth of Confirmed Cases

The chart bellow shows the evolution of cases day-by-day and an rolling average of 14 days (2 weeks).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


data['confirmed_avg2w'] = data['NewCases'].rolling(window=14, center=False).mean()
#data.set_index('Date',inplace=True)
#data.head(20)

# https://stackoverflow.com/a/49714879/1388312
def hide_xticks(ax, every_nth):
    for n, label in enumerate(ax.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)

fig, ax = plt.subplots(figsize=(16,9))
fig.patch.set_facecolor('white')
sns.set_style('whitegrid')

g1 = sns.lineplot(data=data,
             y='confirmed_avg2w',
             x=data.index,
             ax=ax,
             color='red',
             label='Confirmed cases (rolling avg 2w)',
             linewidth=2)

#bar_colors = ['skyblue' if (x < data['NewCases'].max()) else 'orange' for x in data['NewCases'].values ]

g2 = sns.barplot(data=data,
            y='NewCases',
            x=data.index,
            ax=ax,
            #palette=bar_colors,
            color='skyblue',
            label='Confirmed cases')

ax.set_xticklabels(labels=data['Date'].apply(lambda d: d.strftime('%d/%m')),
                   rotation=90)

hide_xticks(ax, 7) # hide every # ticks
            
ax.set_title('COVID in Limeira-SP, Brazil - Confirmed cases - Daily growth', fontsize=16)
ax.set_ylabel('Number of cases', fontsize=12)
ax.set_xlabel('Date', fontsize=12)

ax.legend(loc='upper right')

plt.show()

In [ ]:
# import plotly.graph_objs as go

# # Creating trace1
# trace1 = go.Scatter(
#                     x = df.world_rank,
#                     y = df.citations,
#                     mode = "lines",
#                     name = "citations",
#                     marker = dict(color = 'rgba(16, 112, 2, 0.8)'))

                   
# data = [trace1]
# layout = dict(title = 'COVID in Limeira-SP, Brazil - Confirmed cases - Daily growth',
#               xaxis= dict(title= 'Date',ticklen= 5,zeroline= False)
#              )
# fig = dict(data = data, layout = layout)
# iplot(fig)


## Weekly Growth vs Confirmed

The next chart shows the total number of new cases per week. Also, there is a line with the total number of confirmed cases by the end of that week. Before, the table shows up the confirmed cases for the last weeks.

In [ ]:
# create the 'week' column
weeks = []
def getweek(d):
    key = int(str(d.year) + str(d.isocalendar()[1]))
    if (key not in weeks):
        weeks.append(key)        
    return weeks.index(key) + 1

data['week'] = data['Date'].apply(getweek)

# get weekly data
weekly = data[['Confirmed', 'NewCases', 'week']].groupby(by='week').agg({'Confirmed': 'max', 'NewCases': 'sum'}).reset_index()
weekly.tail(5)

In [ ]:
def show_values(column, data, g, offset=5, color='royalblue', use_int=True):
    for index, row in data.iterrows():
        g.text(row.name, row[column]+offset, (int(row[column]) if use_int else round(row[column],2)) if not np.isnan(row[column]) else 0, color=color, ha="center")

fig, ax = plt.subplots(figsize=(16,9))
fig.patch.set_facecolor('white')
sns.set_style('whitegrid')

g = sns.barplot(data=weekly,
                y='NewCases',
                x=weekly.index,
                ax=ax,
                #palette=bar_colors,
                color='royalblue',
                label='New cases per week')

show_values('NewCases', weekly, g)
        
ax2 = ax.twinx()
ax2.grid(None)
ax2.set_ylim([0,weekly['Confirmed'].max()+1000])

sns.pointplot(data=weekly,
             y='Confirmed',
             x=weekly.index,
             ax=ax2,
             color='red',
             label='Confirmed cases',
             scale=0.8)

ax.set_xticklabels(labels=weekly['week'] )

#hide_xticks(ax, 5)
            
ax.set_title('COVID in Limeira,SP,Brazil - Weekly growth vs Total confirmed cases', fontsize=16)
ax.set_ylabel('Number of cases', fontsize=12)
ax2.set_ylabel('Number of cases', fontsize=12)
ax.set_xlabel('Week of pandemic', fontsize=12)

#ax.legend(loc='upper right')

plt.show()

## Weekly Deaths and URC Occupancy

The chart bellow shows the number of COVID deaths per week and the mean URC occupancy (percentage) in that week. Before the chart, there is a table that shows this data for the latest weeks.

In [ ]:
weekly_deaths = data[['Deaths', 'NewDeaths', 'URCOccupancy', 'week']].groupby(by='week').agg({'Deaths': 'max', 'NewDeaths': 'sum', 'URCOccupancy' : 'mean'}).reset_index()
weekly_deaths.tail()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
fig.patch.set_facecolor('white')
sns.set_style('whitegrid')
ax.set_ylim([0,60])

g = sns.barplot(data=weekly_deaths,
                y='NewDeaths',
                x=weekly_deaths.index,
                ax=ax,
                #palette=bar_colors,
                color='darkorange',
                label='New deaths per week')

show_values('NewDeaths', weekly_deaths, g, offset=0.25, color='darkorange')
        
ax2 = ax.twinx()
ax2.grid(None)
ax2.set_ylim([0,1])

g2 = sns.pointplot(data=weekly_deaths,
                 y='URCOccupancy',
                 x=weekly_deaths.index,
                 ax=ax2,
                 color='darkblue',
                 label='URC Occupancy',
                 scale=0.8)

show_values('URCOccupancy', weekly_deaths, g2, offset=0.01, color='darkblue', use_int=False)

ax.set_xticklabels(labels=weekly_deaths['week'])

#hide_xticks(ax, 5)
            
ax.set_title('COVID in Limeira,SP,Brazil - Weekly deaths vs URC Occpupancy', fontsize=16)
ax.set_ylabel('Number of deaths', fontsize=12)
ax2.set_ylabel('URC Occupancy', fontsize=12)
ax.set_xlabel('Week of pandemic', fontsize=12)

ax.legend(loc='upper right')

plt.show()